In [2]:
import os
import sys
import re
import numpy as np
import pandas as pd
import scipy.io as sio
import matplotlib.pyplot as plt
from scipy.stats import kurtosis, skew
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.neural_network import MLPClassifier


In [3]:
#Importing the data
datasub2 = sio.loadmat('Long_Words/sub_2b_ch64_l_eog_removed_256Hz.mat')['eeg_data_wrt_task_rep_no_eog_256Hz_last_beep']
datasub3 = sio.loadmat('Long_Words/sub_3b_ch80_l_eog_removed_256Hz.mat')['eeg_data_wrt_task_rep_no_eog_256Hz_last_beep']
datasub6 = sio.loadmat('Long_Words/sub_6_ch64_l_eog_removed_256Hz.mat')['eeg_data_wrt_task_rep_no_eog_256Hz_last_beep']
datasub7 = sio.loadmat('Long_Words/sub_7_ch64_l_eog_removed_256Hz.mat')['eeg_data_wrt_task_rep_no_eog_256Hz_last_beep']
datasub9 = sio.loadmat('Long_Words/sub_9c_ch64_l_eog_removed_256Hz.mat')['eeg_data_wrt_task_rep_no_eog_256Hz_last_beep']
datasub2 = sio.loadmat('Long_Words/sub_11b_ch64_l_eog_removed_256Hz.mat')['eeg_data_wrt_task_rep_no_eog_256Hz_last_beep']

In [ ]:


def compute_time_domain_features(eeg_data):
    time_features = {}

    # Mean
    time_features['mean'] = np.mean(eeg_data)

    # Standard deviation
    time_features['std'] = np.std(eeg_data)

    # Kurtosis
    time_features['kurtosis'] = kurtosis(eeg_data)

    # Energy
    time_features['Energy'] = np.sum(np.square(eeg_data))

    # RMS (Root Mean Square)
    time_features['RMS'] = np.sqrt(np.mean(np.square(eeg_data)))

    # Zero-crossing rate
    zero_crossings = np.where(np.diff(np.sign(eeg_data)))[0]
    zero_crossing_rate = len(zero_crossings) / (len(eeg_data) - 1)
    time_features['Zero-crossing rate'] = zero_crossing_rate

    # Hjorth parameters
    # Hjorth parameters
    time_features['hjorth_activity'] = np.sqrt(time_features['Energy'] / time_features['std']**2)
    time_features['hjorth_mobility'] = time_features['mean'] / time_features['std']
    time_features['hjorth_complexity'] = time_features['kurtosis'] / (time_features['std']**4)

    # Skewness
    time_features['Skewness'] = skew(eeg_data)

    # Median
    time_features['Median'] = np.median(eeg_data)


    # Range
    time_features['Range'] = np.ptp(eeg_data)

    # Inter-quartile range
    time_features['Inter-quartile range'] = np.percentile(eeg_data, 75) - np.percentile(eeg_data, 25)

    # Variance
    time_features['Variance'] = np.var(eeg_data)

    # # Autocorrelation
    # autocorr = np.correlate(eeg_data, eeg_data, mode='full')
    # time_features['Autocorrelation'] = autocorr[len(autocorr)//2:]


    return time_features

In [ ]:
def extract_features(eeg_data):
  channels_to_select = [i for i in range(64) if i not in [0, 9, 32, 63]]  # Channels to select (excluding 0, 9, 32, and 63)
  data=eeg_data[channels_to_select,:]
  features=[]

  for i in range(60):
    results=compute_time_domain_features(data[i,:])
    features.extend(list(results.values()))
    

    
  return features




In [ ]:
# function for data augmentation
def data_augmentation(data):
    # Find the shape of the data
    n,m=data.shape
    # Initialize the array
    result=np.empty((0,840))
    j=0
    
    # Loop through the data with a stride of 64 samples
    for i in range(0, 1280, 64):
    # Select a window of 256 samples from the data, starting at index i
        window = data[:, i:i+256]
        features=extract_features(window)
        
        result=np.vstack((result, (np.array(features)).reshape((1,-1))))
        

        # Stop the loop if i+256 is greater than or equal to 1280
        if i+256 >= 1280:
            break
    
    
        
    return result

In [ ]:

def Augmentation_feature(data):
    # find the dimension
    n,m=data.shape
    #iterating over words

    feature= np.empty((0,841))
    #Iterating over word type
    for i in range(n):
        #iterating over trials
        for j in range(m):
            # applying the data augmentation function

            window=data_augmentation(data[i][j])
            if i==0:
              label=np.zeros((17,1))
              window=np.hstack((window, label))
            else:
              label=np.ones((17,1))
              window=np.hstack((window, label))
            feature=np.vstack((feature, window))
          

    return feature

In [ ]:
def calculate_performance(y_test, y_pred):
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    print(f'accuracy: {accuracy}, precision: {precision}, recall: {recall}, f1 {f1}')



In [ ]:
def train_model(eeg_data):
  data=Augmentation_feature(eeg_data)
  features=data[:,:-1]
  label=data[:,-1]
    
  # Example usage
  X_train, X_test, y_train, y_test = train_test_split(features, label, test_size=0.2, random_state=42)
  scaler = StandardScaler()
  train_data = scaler.fit_transform(X_train)
  test_data = scaler.transform(X_test)
  n_components =   200# Specify the number of components you want to keep

  pca = PCA(n_components=n_components)
  train_pca = pca.fit_transform(train_data)
  test_pca = pca.transform(test_data)
  y_train=y_train.astype(int)
  y_test=y_test.astype(int)
  
  # Import other classifiers as needed

  # Train classifiers with different n_components values
  clf1 = SVC()
  clf1.fit(train_pca, y_train)
  y_pred_pca = clf1.predict(test_pca)
  print("pca linear performance: ")
  calculate_performance(y_test, y_pred_pca)

  
  clf2 = RandomForestClassifier()
  clf2.fit(train_pca, y_train)
  y_pred_rfc = clf2.predict(test_pca)
  print("Random Forest performance: ")
  calculate_performance(y_test, y_pred_rfc)

  model = MLPClassifier(hidden_layer_sizes=(100,), activation='relu', solver='adam', random_state=42)
  model.fit(train_pca, y_train)
  y_pred = model.predict(test_pca)
  y_pred_mlp = [round(value) for value in y_pred]
  print('MLP performance: ')
  calculate_performance(y_test, y_pred_mlp)















In [ ]:
train_model(datasub2)

pca linear performance: 
accuracy: 0.6617647058823529, precision: 0.6270718232044199, recall: 0.7049689440993789, f1 0.6637426900584795
Random Forest performance: 
accuracy: 0.7, precision: 0.6798780487804879, recall: 0.6925465838509317, f1 0.6861538461538462
MLP performance: 
accuracy: 0.7955882352941176, precision: 0.771513353115727, recall: 0.8074534161490683, f1 0.7890743550834598


In [ ]:
train_model(datasub3)

pca linear performance: 
accuracy: 0.6691176470588235, precision: 0.6343490304709142, recall: 0.7111801242236024, f1 0.6705710102489019
Random Forest performance: 
accuracy: 0.6941176470588235, precision: 0.6676470588235294, recall: 0.7049689440993789, f1 0.6858006042296072
MLP performance: 
accuracy: 0.8529411764705882, precision: 0.846875, recall: 0.8416149068322981, f1 0.8442367601246105


In [ ]:
train_model(datasub6)

pca linear performance: 
accuracy: 0.6441176470588236, precision: 0.6136363636363636, recall: 0.6708074534161491, f1 0.6409495548961425
Random Forest performance: 
accuracy: 0.7323529411764705, precision: 0.7121212121212122, recall: 0.7298136645962733, f1 0.7208588957055214
MLP performance: 
accuracy: 0.7970588235294118, precision: 0.7857142857142857, recall: 0.7857142857142857, f1 0.7857142857142857


In [ ]:
train_model(datasub9)

pca linear performance: 
accuracy: 0.7823529411764706, precision: 0.7604790419161677, recall: 0.7888198757763976, f1 0.7743902439024389
Random Forest performance: 
accuracy: 0.763235294117647, precision: 0.7388724035608308, recall: 0.7732919254658385, f1 0.7556904400606981
MLP performance: 
accuracy: 0.8632352941176471, precision: 0.8438438438438438, recall: 0.8726708074534162, f1 0.8580152671755725


In [ ]:
train_model(datasub7)

pca linear performance: 
accuracy: 0.8073529411764706, precision: 0.7800586510263929, recall: 0.8260869565217391, f1 0.8024132730015083
Random Forest performance: 
accuracy: 0.763235294117647, precision: 0.7588424437299035, recall: 0.7329192546583851, f1 0.7456556082148499
MLP performance: 
accuracy: 0.8764705882352941, precision: 0.8628048780487805, recall: 0.8788819875776398, f1 0.8707692307692306


In [ ]:
train_model(datasub11)

pca linear performance: 
accuracy: 0.6544117647058824, precision: 0.6211699164345403, recall: 0.6925465838509317, f1 0.6549192364170338
Random Forest performance: 
accuracy: 0.7044117647058824, precision: 0.673352435530086, recall: 0.7298136645962733, f1 0.7004470938897168
MLP performance: 
accuracy: 0.7970588235294118, precision: 0.772189349112426, recall: 0.8105590062111802, f1 0.7909090909090909


In [ ]:
'''
pip install xgboost
import xgboost as xgb
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)
params = {
    'objective': 'binary:logistic',  # for binary classification
    'eval_metric': 'logloss',  # evaluation metric
    'max_depth': 3,  # maximum depth of a tree
    'eta': 0.1,  # learning rate
    'subsample': 0.8,  # subsample ratio of the training instances
    'colsample_bytree': 0.8  # subsample ratio of columns when constructing each tree
}
num_rounds = 100  # number of boosting rounds
model = xgb.train(params, dtrain, num_rounds)
y_pred = model.predict(dtest)
y_pred_labels = [round(value) for value in y_pred]
accuracy = accuracy_score(y_test, y_pred_labels)
'''

"\npip install xgboost\nimport xgboost as xgb\ndtrain = xgb.DMatrix(X_train, label=y_train)\ndtest = xgb.DMatrix(X_test, label=y_test)\nparams = {\n    'objective': 'binary:logistic',  # for binary classification\n    'eval_metric': 'logloss',  # evaluation metric\n    'max_depth': 3,  # maximum depth of a tree\n    'eta': 0.1,  # learning rate\n    'subsample': 0.8,  # subsample ratio of the training instances\n    'colsample_bytree': 0.8  # subsample ratio of columns when constructing each tree\n}\nnum_rounds = 100  # number of boosting rounds\nmodel = xgb.train(params, dtrain, num_rounds)\ny_pred = model.predict(dtest)\ny_pred_labels = [round(value) for value in y_pred]\naccuracy = accuracy_score(y_test, y_pred_labels)\n"

In [ ]:
'''
pca = PCA()
pca.fit(train_data)

cumulative_variance_ratio = np.cumsum(pca.explained_variance_ratio_)
plt.plot(range(1, len(cumulative_variance_ratio) + 1), cumulative_variance_ratio)
plt.xlabel('Number of Components')
plt.ylabel('Cumulative Explained Variance Ratio')
plt.show()
'''

"\npca = PCA()\npca.fit(train_data)\n\ncumulative_variance_ratio = np.cumsum(pca.explained_variance_ratio_)\nplt.plot(range(1, len(cumulative_variance_ratio) + 1), cumulative_variance_ratio)\nplt.xlabel('Number of Components')\nplt.ylabel('Cumulative Explained Variance Ratio')\nplt.show()\n"

In [ ]:
import numpy as np
from pywt import wavedec, waverec
from scipy.optimize import minimize

def radwt_features(eeg_signal, wavelet='db4', levels=4, p=2, q=2, s=0):
  """
  This function finds features using RADWT on an EEG signal.

  Args:
    eeg_signal (np.array): The EEG signal.
    wavelet (str): The wavelet to use.
    levels (int): The number of levels of wavelet decomposition.
    p (int): The number of vanishing moments.
    q (int): The number of vanishing moments of the detail coefficients.
    s (int): The number of vanishing moments of the approximation coefficients.

  Returns:
    A list of features.
  """

  # Get the wavelet coefficients.
  wavelet_coeffs = wavedec(eeg_signal, wavelet, levels, p, q, s)

  # Extract the energy of each level of wavelet coefficients.
  energy = np.array([np.sum(coeff**2) for coeff in wavelet_coeffs])

  # Return the energy of each level of wavelet coefficients as features.
  return energy

def objective_function(params, eeg_signal):
  """
  This function defines the objective function for PSO.

  Args:
    params (list): The PSO parameters.
    eeg_signal (np.array): The EEG signal.

  Returns:
    The objective function value.
  """

  # Reconstruct the signal using RADWT.
  reconstructed_signal = waverec(eeg_signal, params)

  # Calculate the MSE of the reconstructed signal.
  mse = np.mean((eeg_signal - reconstructed_signal)**2)

  # Return the MSE.
  return mse

def optimize_radwt(eeg_signal):
  """
  This function optimizes RADWT using PSO.

  Args:
    eeg_signal (np.array): The EEG signal.

  Returns:
    The optimal PSO parameters.
  """

  # Initialize the PSO parameters.
  params = [0.5, 0.5, 2, 2, 0]

  # Optimize the RADWT parameters using PSO.
  result = minimize(objective_function, params, args=(eeg_signal,), method='PSO', options={'popsize': 100, 'maxiter': 100})

  # Return the optimal PSO parameters.
  return result.x

if __name__ == '__main__':
  # Load the EEG signal.
  eeg_signal = datasub2[0][0][1]

  # Optimize RADWT using PSO.
  optimal_params = optimize_radwt(eeg_signal)

  # Reconstruct the signal using the optimal RADWT parameters.
  reconstructed_signal = waverec(eeg_signal, optimal_params)

  # Get the wavelet coefficients of the reconstructed signal.
  wavelet_coeffs = wavedec(reconstructed_signal, 'db4')

  # Extract the energy of each level of wavelet coefficients.
  energy = np.array([np.sum(coeff**2) for coeff in wavelet_coeffs])

  # Print the features.
  print(energy)

  # Print the number of levels of wavelet decomposition used.
  print(optimal_params[0])

ValueError: ignored

In [ ]:
pip install pyswarms

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 7.4 MB/s eta 0:00:00


In [ ]:
from pyswarms.single.global_best import GlobalBestPSO
eeg_signal=datasub2[0][0][1]
def objective_function():
  """
  This function defines the objective function for PSO.

  Args:
    params (list): The PSO parameters.
    eeg_signal (np.array): The EEG signal.

  Returns:
    The objective function value.
  """


  # Reconstruct the signal using RADWT.
  reconstructed_signal = waverec( 'db4', level=4)

  # Calculate the MSE of the reconstructed signal.
  mse = np.mean((eeg_signal - reconstructed_signal)**2)

  # Return the MSE.
  return mse

def optimize_radwt(eeg_signal):
  """
  This function optimizes RADWT using PSO.

  Args:
    eeg_signal (np.array): The EEG signal.

  Returns:
    The optimal PSO parameters.
  """

  # Initialize the PSO parameters.
  options = {'c1': 1.3, 'c2': 1.3, 'w': 0.9}

  # Create the PSO optimizer.
  optimizer = GlobalBestPSO(n_particles=150, dimensions=4,  options=options)

  # Optimize the RADWT parameters using PSO.
  result = optimizer.optimize(objective_function, iters=200)

  # Return the optimal PSO parameters.
  return result.x

if __name__ == '__main__':
  # Load the EEG signal.
  eeg_signal = datasub2[0][0][1]

  # Optimize RADWT using PSO.
  optimal_params = optimize_radwt(eeg_signal)

  # Reconstruct the signal using the optimal RADWT parameters.
  reconstructed_signal = waverec(optimal_params)

  # Get the wavelet coefficients of the reconstructed signal.
  wavelet_coeffs = wavedec(reconstructed_signal, 'db4')

  # Extract the energy of each level of wavelet coefficients.
  energy = np.array([np.sum(coeff**2) for coeff in wavelet_coeffs])

  # Print the features.
  print(energy)

  # Print the number of levels of wavelet decomposition used.
  print(optimal_params[0])

2023-05-15 10:15:22,083 - pyswarms.single.global_best - INFO - Optimize for 200 iters with {'c1': 1.3, 'c2': 1.3, 'w': 0.9}
pyswarms.single.global_best:   0%|          |0/200


TypeError: ignored

In [ ]:
eeg_signal=datasub2[0][0][1]

In [ ]:
eeg_signal

array([-0.60453319, -4.32794179,  0.25399298, ..., -4.86514305,
       -4.48801125, -1.74475553])

In [ ]:
import numpy as np
import pywt
from sklearn.preprocessing import StandardScaler
import pyswarms as ps

# Constants
NUM_CHANNELS = 64
SAMPLING_RATE = 256
DWT_LEVELS = 5
NUM_PARTICLES = 30
MAX_ITERATIONS = 50

# Define the objective function (fitness function) to be optimized
def objective_function(coefficients, data):
    # Apply RADWT on data using the given coefficients
    radwt_output = pywt.waverec(coefficients, 'db4')

    # Extract features from the RADWT output
    # You can customize this part based on the specific features you want to extract
    # Here, we're calculating the mean and standard deviation for each channel
    features = []
    for channel_data in radwt_output:
        channel_features = [
            np.mean(channel_data),
            np.std(channel_data)
        ]
        features.extend(channel_features)

    return features

# Define the Particle Swarm Optimization
def pso_optimization(data):
    # Normalize the data


    # Define the bounds for the particle positions
    bounds = ([-1] * NUM_CHANNELS * DWT_LEVELS, [1] * NUM_CHANNELS * DWT_LEVELS)

    # Define the options for the optimizer
    options = {'c1': 2.0, 'c2': 2.0, 'w': 0.7}

    # Define the optimizer
    optimizer = ps.single.GlobalBestPSO(n_particles=NUM_PARTICLES, dimensions=NUM_CHANNELS * DWT_LEVELS,
                                        options=options, bounds=bounds)

    # Perform optimization
    best_position, _ = optimizer.optimize(objective_function, iters=MAX_ITERATIONS, data=data)

    # Decode the best position into RADWT coefficients
    best_coefficients, _ = pywt.coeffs_to_array(best_position, shape=(NUM_CHANNELS, DWT_LEVELS))

    return best_coefficients

# Example usage
eeg_data = ...  # Your 5-second EEG data with shape (NUM_CHANNELS, SAMPLING_RATE * 5)
radwt_coefficients = pso_optimization(eeg_data)

2023-05-15 10:08:11,069 - pyswarms.single.global_best - INFO - Optimize for 50 iters with {'c1': 2.0, 'c2': 2.0, 'w': 0.7}
pyswarms.single.global_best:   0%|          |0/50


ValueError: ignored